In [34]:
import json
import re
import os
os.environ['OPENAI_API_KEY']=""

In [35]:
f = open('news.article.json')
data = json.load(f)
docs=""
# Iterating through the json
# list
for i in data:
    s=i['articleBody']
    s=s.replace("\'s", "'s")
    s = re.sub(r'\n+', ' ', s)
    #s=s.replace('\s',"'")
    s.replace(r'\"','"')
    docs+=s

In [36]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=1000)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [39]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=100)
chunks = text_splitter.split_text(docs)
embeddings = OpenAIEmbeddings()

In [40]:
vectordb = FAISS.from_texts(chunks[:500], embeddings)
retriever = vectordb.as_retriever()

In [41]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"vector_search",
                      "Search for information about Israel-Hamas War. For any questions about Israel-Hamas War, you must use this tool!")

In [42]:
tools=[wiki,retriever_tool]

In [43]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [44]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [45]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [46]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [47]:
agent_executor.invoke({"input":"What happend at the AI-Shifa Hospital"})



> Entering new AgentExecutor chain...

Invoking: `vector_search` with `{'query': 'AI-Shifa Hospital'}`


that more than one-third of the city’s hospitals are no longer operating and Israel’s bombing continues, health care professionals fear the worst. “The health system here is in its last throes before it completely boots down. If the electricity goes, that’s it. It just becomes a mass grave,” says Dr. Ghassan Abu-Sittah, a British-Palestinian plastic and reconstructive surgeon who has been working at Al-Shifa hospital over the last two weeks. “There’s no hospital, if there’s no electricity.” Read More: ‘Our Death Is Pending.’ Stories of Loss and Grief From Gaza Right now, his sense is that it will be “days, rather than weeks,” until Al-Shifa runs out of fuel needed to keep the hospital running. Palestine’s Ministry of Health said Tuesday that hospital generators will stop running in 48 hours, and aid workers tell TIME that the city is expected to run out of fuel on Wednesday evenin

{'input': 'What happend at the AI-Shifa Hospital',
 'output': "The Al-Shifa Hospital in Gaza has been facing a critical situation due to the ongoing conflict between Israel and Hamas. The hospital is struggling to operate as it is running out of fuel needed to keep essential services running. The electricity supply is at risk, which is crucial for the hospital's operations, especially for neonatal babies dependent on ventilators. The hospital is overcrowded, operating well beyond its capacity, and facing shortages of medical supplies and equipment.\n\nThe conflict has resulted in a high number of casualties, with more than 6,400 people killed and over 17,000 injured in Gaza. The hospitals, including Al-Shifa, are overwhelmed with patients, and the healthcare system is on the brink of collapse. The situation is exacerbated by the lack of clean water, inadequate sanitation, and the risk of infectious diseases spreading in the crowded hospital environment.\n\nDespite the challenges, healt

In [48]:
agent_executor.invoke({"input":"What is FIFA"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `FIFA`


Page: FIFA
Summary: The Fédération Internationale de Football Association (French for 'International Federation of Association Football'; abbreviated as FIFA and pronounced in English as  FEE-fə) is an international self-regulatory governing body of association football, beach soccer, and futsal. It was founded in 1904 to oversee international competition among the national associations of Belgium, Denmark, France, Germany, the Netherlands, Spain (represented by the Madrid Football Club), Sweden, and Switzerland. Headquartered in Zürich, Switzerland, its membership now comprises 211 national associations. These national associations must also be members of one of the six regional confederations into which the world is divided: CAF (Africa), AFC (Asia and Australia), UEFA (Europe), CONCACAF (North & Central America and the Caribbean), OFC (Oceania), and CONMEBOL (South America).
FIFA outlines several objectives

{'input': 'What is FIFA',
 'output': 'FIFA stands for Fédération Internationale de Football Association, which is the international governing body of association football, beach soccer, and futsal. It was founded in 1904 and is headquartered in Zurich, Switzerland. FIFA oversees international competitions among national football associations and has 211 member associations. The organization is divided into six regional confederations: CAF (Africa), AFC (Asia and Australia), UEFA (Europe), CONCACAF (North & Central America and the Caribbean), OFC (Oceania), and CONMEBOL (South America). FIFA has various objectives outlined in its organizational statutes, including growing association football internationally.'}